# Building Structured Agents with Pydantic AI and DeepSeek

In [1]:
!pip -q install pydantic-ai
!pip -q install nest_asyncio
!pip -q install devtools
!pip -q install tavily-python

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-core 0.12.1 requires pydantic<2.10.0,>=2.7.0, but you have pydantic 2.10.6 which is incompatible.
llama-index-core 0.12.1 requires tenacity!=8.4.0,<9.0.0,>=8.2.0, but you have tenacity 9.0.0 which is incompatible.
llama-index-legacy 0.9.48.post4 requires tenacity<9.0.0,>=8.2.0, but you have tenacity 9.0.0 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
unstructured-client 0.28.0 requires pydantic<2.10.0,>=2.9.2, but you have pydantic 2.10.6 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### **Restart the notebook**

You will need to restart the notebook after you've installed Pydantic AI and the other dependencies above.

In [ ]:
import os
from google.colab import userdata
from IPython.display import display, Markdown

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_AI_STUDIO')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["DEEPSEEK_API_KEY"] = userdata.get('DEEPSEEK_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY') # If you dont have this use the DDGS alternative below

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## Setting Up search Tavily


### Tavily Search

In [ ]:
from tavily import TavilyClient, AsyncTavilyClient

# Setup the Tavily Client
tavily_client = AsyncTavilyClient(api_key=os.environ["TAVILY_API_KEY"])

# Simple Search
response = await tavily_client.search("What is DeepSeekR1?", max_results=3)

print(response['results'])

[{'title': 'DeepSeek-R1: In-Depth Review and Usage Guide', 'url': 'https://neuroflash.com/blog/deepseek-r1/', 'content': "Helpcenter Access guides and support on how to use neuroflash's AI tools. Free Guides & Templates Download free documents with knowledge on AI content creation. DALL-E 3, GPT-4, Brand Voices, file upload and more - integrated in the neuroflash AI Marketing Content Suite DALL-E 3, GPT-4, Brand Voices, file upload and more - integrated in the neuroflash AI Marketing Content Suite DeepSeek-R1-Lite-Preview is an AI model developed by the Chinese company DeepSeek, similar to ChatGPT. While DeepSeek-R1 excels in reasoning capabilities, when it comes to AI text generation and personalization, neuroflash’s ChatFlash emerges as a top alternative of all GPT models. DeepSeek-R1 is poised to revolutionize the way we understand AI reasoning models, offering superior performance, transparency, and versatility.", 'score': 0.9035075, 'raw_content': None}, {'title': "DeepSeek Unveil

In [ ]:
# Responses
for result in response['results']:
    print(result['title'])
    print(result['content'])
    print('\n')

DeepSeek-R1: In-Depth Review and Usage Guide
Helpcenter Access guides and support on how to use neuroflash's AI tools. Free Guides & Templates Download free documents with knowledge on AI content creation. DALL-E 3, GPT-4, Brand Voices, file upload and more - integrated in the neuroflash AI Marketing Content Suite DALL-E 3, GPT-4, Brand Voices, file upload and more - integrated in the neuroflash AI Marketing Content Suite DeepSeek-R1-Lite-Preview is an AI model developed by the Chinese company DeepSeek, similar to ChatGPT. While DeepSeek-R1 excels in reasoning capabilities, when it comes to AI text generation and personalization, neuroflash’s ChatFlash emerges as a top alternative of all GPT models. DeepSeek-R1 is poised to revolutionize the way we understand AI reasoning models, offering superior performance, transparency, and versatility.


DeepSeek Unveils DeepSeek-R1 RL Models: What's New and How It is better ...
In a recent innovative announcement, Chinese AI lab DeepSeek (which r

In [ ]:
# RAG Context search

# Context Search
context = await tavily_client.get_search_context(query="What is DeepSeekR1?", max_results=3)

print(context)

"[\"{\\\"url\\\": \\\"https://autogpt.net/new-chinese-ai-model-deepseek-r1-can-think-like-humans/\\\", \\\"content\\\": \\\"New Chinese AI Model, DeepSeek-R1, Can \\\\u201cThink\\\\u201d Like Humans AI AI Tools New Chinese AI Model, DeepSeek-R1, Can \\\\u201cThink\\\\u201d Like Humans The competition in the AI world is heating up, and this week, the spotlight is on DeepSeek, a Chinese AI research company that just unveiled DeepSeek-R1, a reasoning AI model designed to rival OpenAI\\\\u2019s o1. Unlike traditional AI models, which often rely on brute-force computations and statistical patterns, reasoning models like DeepSeek-R1 take a more thoughtful approach. DeepSeek-R1 represents a significant step forward for reasoning models and reflects the intensifying competition in the global AI landscape. AI Tools AI Tools AI AI AI Models AI Tools AI AI Tools AI AI Tools AI AI Tools\\\"}\", \"{\\\"url\\\": \\\"https://www.deeplearning.ai/the-batch/deepseek-r1-a-transparent-challenger-to-openai

## Setting up the Agent & Tools

In [ ]:
from __future__ import annotations as _annotations

import asyncio
import os
from dataclasses import dataclass
from typing import Any

from devtools import debug
from httpx import AsyncClient
import datetime

from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic import BaseModel, Field

### Setting up DeepSeek

In [ ]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel

# DeepSeekv3
deepseek_chat_model = OpenAIModel(
    'deepseek-chat',
    base_url='https://api.deepseek.com',
    api_key=os.environ["DEEPSEEK_API_KEY"],
)

# DeepSeekR1
deepseek_reasoner_model = OpenAIModel(
    'deepseek-reasoner',
    base_url='https://api.deepseek.com',
    api_key=os.environ["DEEPSEEK_API_KEY"],
)

In [ ]:
@dataclass
class SearchDataclass:
    max_results: int
    todays_date: str

@dataclass
class ResearchDependencies:
    todays_date: str

class ResearchResult(BaseModel):
    research_title: str = Field(description='This is a top level Markdown heading that covers the topic of the query and answer prefix it with #')
    research_main: str = Field(description='This is a main section that provides answers for the query and research')
    research_bullets: str = Field(description='This is a set of bulletpoints that summarize the answers for query')

In [ ]:
## Make the agent
search_agent = Agent(deepseek_chat_model,
                     deps_type=ResearchDependencies,
                     result_type=ResearchResult,
                     system_prompt='Your a helpful research assistant, you are an expert in research '
                     'If you are given a question you write strong keywords to do 3-5 searches in total '
                     '(each with a query_number) and then combine the results' )


In [ ]:
@search_agent.tool #Tavily
async def get_search(search_data:RunContext[SearchDataclass],query: str, query_number: int) -> dict[str, Any]:
    """Get the search for a keyword query.

    Args:
        query: keywords to search.
    """
    print(f"Search query {query_number}: {query}")
    max_results = search_data.deps.max_results
    results = await tavily_client.get_search_context(query=query, max_results=max_results)

    return results


In [ ]:
## set up the dependencies

# Get the current date
current_date = datetime.date.today()

# Convert the date to a string
date_string = current_date.strftime("%Y-%m-%d")


deps = SearchDataclass(max_results=3, todays_date=date_string)



In [ ]:
result = await search_agent.run(
    'can you give me a very detailed breakdown of the DeepSeekR1 model', deps=deps
)

Search query 1: DeepSeekR1 model architecture
Search query 2: DeepSeekR1 model training process
Search query 3: DeepSeekR1 model applications
Search query 4: DeepSeekR1 model performance benchmarks


In [ ]:
print(result.data)

research_title='# DeepSeek-R1 Model: A Comprehensive Breakdown' research_main="The DeepSeek-R1 model is a state-of-the-art large language model developed by DeepSeek, a Chinese AI lab. It is a 671B parameter Mixture-of-Experts (MoE) model with 37B activated parameters per token. The model is designed to excel in reasoning tasks, particularly in areas such as coding, mathematics, science, and logic reasoning. DeepSeek-R1 has been trained using large-scale reinforcement learning (RL) to enhance its reasoning capabilities.\n\n### Architecture\nDeepSeek-R1 is built on a Mixture-of-Experts (MoE) architecture, which allows it to activate only a subset of its parameters (37B) per token, making it computationally efficient. This architecture enables the model to handle complex reasoning tasks effectively.\n\n### Training Process\nThe training process of DeepSeek-R1 involves a combination of pure reinforcement learning (RL) and iterative fine-tuning. The initial model, DeepSeek-R1-Zero, was tra

In [ ]:
result.data.research_title = '#'+result.data.research_title
print(result.data.research_title)

## DeepSeek-R1 Model: A Comprehensive Breakdown


In [ ]:
print(result.data.research_main)

The DeepSeek-R1 model is a state-of-the-art large language model developed by DeepSeek, a Chinese AI lab. It is a 671B parameter Mixture-of-Experts (MoE) model with 37B activated parameters per token. The model is designed to excel in reasoning tasks, particularly in areas such as coding, mathematics, science, and logic reasoning. DeepSeek-R1 has been trained using large-scale reinforcement learning (RL) to enhance its reasoning capabilities.

### Architecture
DeepSeek-R1 is built on a Mixture-of-Experts (MoE) architecture, which allows it to activate only a subset of its parameters (37B) per token, making it computationally efficient. This architecture enables the model to handle complex reasoning tasks effectively.

### Training Process
The training process of DeepSeek-R1 involves a combination of pure reinforcement learning (RL) and iterative fine-tuning. The initial model, DeepSeek-R1-Zero, was trained using pure RL without supervised fine-tuning, which is a unique approach compare

In [ ]:
print(result.data.research_bullets)

- **Architecture**: DeepSeek-R1 is a 671B parameter Mixture-of-Experts (MoE) model with 37B activated parameters per token.
- **Training Process**: Trained using large-scale reinforcement learning (RL) and iterative fine-tuning, starting with pure RL in DeepSeek-R1-Zero.
- **Applications**: Excels in coding, mathematics, science, and logic reasoning tasks.
- **Performance Benchmarks**: Matches OpenAI’s o1 in chemistry, mathematics, and coding; achieves state-of-the-art results on MMLU, MMLU-Pro, GPQA, and IFEval benchmarks.
- **Distilled Models**: Includes distilled versions like DeepSeek-R1-Distill-Qwen-32B, fine-tuned from open-source models for computational efficiency.
- **Open Source**: Released as an open-weight model, fostering research and innovation.
- **Cost Efficiency**: Approximately 27.4x cheaper than OpenAI’s o1 for input and output tokens.


In [ ]:

combined_markdown = "\n\n".join([result.data.research_title, result.data.research_main, result.data.research_bullets])

Markdown(combined_markdown)

## DeepSeek-R1 Model: A Comprehensive Breakdown

The DeepSeek-R1 model is a state-of-the-art large language model developed by DeepSeek, a Chinese AI lab. It is a 671B parameter Mixture-of-Experts (MoE) model with 37B activated parameters per token. The model is designed to excel in reasoning tasks, particularly in areas such as coding, mathematics, science, and logic reasoning. DeepSeek-R1 has been trained using large-scale reinforcement learning (RL) to enhance its reasoning capabilities.

### Architecture
DeepSeek-R1 is built on a Mixture-of-Experts (MoE) architecture, which allows it to activate only a subset of its parameters (37B) per token, making it computationally efficient. This architecture enables the model to handle complex reasoning tasks effectively.

### Training Process
The training process of DeepSeek-R1 involves a combination of pure reinforcement learning (RL) and iterative fine-tuning. The initial model, DeepSeek-R1-Zero, was trained using pure RL without supervised fine-tuning, which is a unique approach compared to other models. The training pipeline includes cold-start data and iterative fine-tuning, which helps in enhancing the model's reasoning capabilities.

### Applications
DeepSeek-R1 has a wide range of applications, particularly in fields that require advanced reasoning capabilities. It has been used in tasks such as coding, mathematics, science, and logic reasoning. The model's open-weight nature allows researchers to study and build on the algorithm, making it a valuable tool for the research community.

### Performance Benchmarks
DeepSeek-R1 has shown impressive performance on various benchmarks. It performs reasoning tasks at the same level as OpenAI’s o1 model, particularly in chemistry, mathematics, and coding. The model has also been benchmarked against other models on tasks such as MMLU, MMLU-Pro, GPQA, and IFEval, where it has achieved state-of-the-art results.

### Distilled Models
To make the reasoning capabilities of DeepSeek-R1 more accessible, the researchers have created distilled versions of the model. These distilled models, such as DeepSeek-R1-Distill-Qwen-32B, are fine-tuned based on open-source models like Qwen and Llama. These distilled models can be utilized in the same manner as their parent models and are designed to be more computationally efficient.

### Open Source and Community Impact
DeepSeek-R1 is released as an open-weight model, meaning that researchers can study and build on the algorithm. This open-source nature fosters collaboration and innovation within the AI research community. The model's API and distilled versions are also available, making it easier for researchers with limited computing power to experiment with the model.

### Cost Efficiency
DeepSeek-R1 is significantly more cost-efficient compared to other models like OpenAI’s o1. It is roughly 27.4x cheaper for input and output tokens, making it an attractive option for researchers and developers.

In summary, DeepSeek-R1 is a powerful and efficient reasoning model that leverages a unique training approach and MoE architecture to achieve state-of-the-art performance in various reasoning tasks. Its open-source nature and cost efficiency make it a valuable tool for the AI research community.

- **Architecture**: DeepSeek-R1 is a 671B parameter Mixture-of-Experts (MoE) model with 37B activated parameters per token.
- **Training Process**: Trained using large-scale reinforcement learning (RL) and iterative fine-tuning, starting with pure RL in DeepSeek-R1-Zero.
- **Applications**: Excels in coding, mathematics, science, and logic reasoning tasks.
- **Performance Benchmarks**: Matches OpenAI’s o1 in chemistry, mathematics, and coding; achieves state-of-the-art results on MMLU, MMLU-Pro, GPQA, and IFEval benchmarks.
- **Distilled Models**: Includes distilled versions like DeepSeek-R1-Distill-Qwen-32B, fine-tuned from open-source models for computational efficiency.
- **Open Source**: Released as an open-weight model, fostering research and innovation.
- **Cost Efficiency**: Approximately 27.4x cheaper than OpenAI’s o1 for input and output tokens.

In [ ]:
debug(result)

## Using DeepSeekR1

In [ ]:
class LifeMeaningStructuredResult(BaseModel):
    life_meaning_title: str = Field(description='This is a top level Markdown heading that covers the topic of the query and answer prefix it with #')
    life_meaning_main: str = Field(description='This is a main section that provides answers for the query and questions')
    life_meaning_bullets: str = Field(description='This is a set of bulletpoints that summarize the answers for query')

## Make the agent
reasoner_agent = Agent(deepseek_reasoner_model,
                     result_type=LifeMeaningStructuredResult,
                     system_prompt='Your a helpful and wise reasoning assistant, you are an expert in thinking '
                     'If you are given a question you think carefully and then respond with a title, your thinking, '
                     'a set of bullets points summary and a final answer '
                      )

In [ ]:
result = await reasoner_agent.run('What is the meaning of life?')

BadRequestError: Error code: 400 - {'error': {'message': 'deepseek-reasoner does not support Function Calling', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}

In [ ]:
## Make the agent
reasoner_agent = Agent(deepseek_reasoner_model,
                    #  deps_type=ResearchDependencies,
                    #  result_type=LifeMeaningStructuredResult,
                     system_prompt='Your a helpful and wise reasoning assistant, you are an expert in thinking '
                     'If you are given a question you think carefully and then respond with a title, your thinking, '
                     'a set of bullets points summary and a final answer '
                       )

In [ ]:
result = await reasoner_agent.run('What is the meaning of life?')

In [ ]:
result.all_messages_json()

b'[{"parts":[{"content":"Your a helpful and wise reasoning assistant, you are an expert in thinking If you are given a question you think carefully and then respond with a title, your thinking, a set of bullets points summary and a final answer ","dynamic_ref":null,"part_kind":"system-prompt"},{"content":"What is the meaning of life?","timestamp":"2025-01-24T09:23:11.724685Z","part_kind":"user-prompt"}],"kind":"request"},{"parts":[{"content":"**Title: The Quest for Meaning: Exploring Life\'s Ultimate Question**  \\n\\n**Thinking Process:**  \\nThe question \\"What is the meaning of life?\\" is inherently subjective and multifaceted. To address it, we must synthesize perspectives from philosophy, religion, science, and individual experience:  \\n1. **Philosophical Views**: Existentialism argues that life\xe2\x80\x99s meaning is self-created (e.g., Sartre), while nihilism posits no inherent meaning. Absurdism (Camus) accepts the tension between seeking meaning and an indifferent universe

In [ ]:
Markdown(result.data)

**Title: The Quest for Meaning: Exploring Life's Ultimate Question**  

**Thinking Process:**  
The question "What is the meaning of life?" is inherently subjective and multifaceted. To address it, we must synthesize perspectives from philosophy, religion, science, and individual experience:  
1. **Philosophical Views**: Existentialism argues that life’s meaning is self-created (e.g., Sartre), while nihilism posits no inherent meaning. Absurdism (Camus) accepts the tension between seeking meaning and an indifferent universe.  
2. **Religious/Spiritual Perspectives**: Many traditions tie purpose to transcendence—e.g., union with the divine (Hinduism), following God’s will (Abrahamic faiths), or achieving enlightenment (Buddhism).  
3. **Scientific Lens**: Biology suggests survival and reproduction, but human consciousness expands this to curiosity, progress, and legacy.  
4. **Individual and Cultural Subjectivity**: Personal fulfillment, relationships, creativity, or societal contribution often define meaning, shaped by cultural norms (e.g., collectivism vs. individualism).  
5. **Modern Psychology**: Positive psychology emphasizes flourishing through purpose, relationships, and engagement (e.g., Seligman’s "PERMA" model).  

**Summary Bullet Points:**  
- No universal answer exists; interpretations vary by context and belief.  
- Philosophy offers frameworks for creating or rejecting inherent meaning.  
- Spirituality often links purpose to a higher reality or moral duty.  
- Biologically, life’s "goal" is propagation, but humans transcend this through abstract goals.  
- Meaning is frequently tied to personal growth, connections, or legacy.  
- The search itself may be integral to the human experience.  

**Final Answer:**  
The meaning of life is not a singular truth but a deeply personal and evolving concept. It can be found in self-defined purpose (philosophy), spiritual fulfillment (religion), contributing to collective progress (society), or simply embracing existence (absurdism). While answers vary, the pursuit of meaning—whether through love, creativity, knowledge, or service—often becomes the journey that defines a life well-lived.

In [ ]:
Markdown(result.data)

**Title: The Quest for Meaning: Exploring Life's Ultimate Question**  

**Thinking Process:**  
The question "What is the meaning of life?" is inherently subjective and multifaceted. To address it, we must synthesize perspectives from philosophy, religion, science, and individual experience:  
1. **Philosophical Views**: Existentialism argues that life’s meaning is self-created (e.g., Sartre), while nihilism posits no inherent meaning. Absurdism (Camus) accepts the tension between seeking meaning and an indifferent universe.  
2. **Religious/Spiritual Perspectives**: Many traditions tie purpose to transcendence—e.g., union with the divine (Hinduism), following God’s will (Abrahamic faiths), or achieving enlightenment (Buddhism).  
3. **Scientific Lens**: Biology suggests survival and reproduction, but human consciousness expands this to curiosity, progress, and legacy.  
4. **Individual and Cultural Subjectivity**: Personal fulfillment, relationships, creativity, or societal contribution often define meaning, shaped by cultural norms (e.g., collectivism vs. individualism).  
5. **Modern Psychology**: Positive psychology emphasizes flourishing through purpose, relationships, and engagement (e.g., Seligman’s "PERMA" model).  

**Summary Bullet Points:**  
- No universal answer exists; interpretations vary by context and belief.  
- Philosophy offers frameworks for creating or rejecting inherent meaning.  
- Spirituality often links purpose to a higher reality or moral duty.  
- Biologically, life’s "goal" is propagation, but humans transcend this through abstract goals.  
- Meaning is frequently tied to personal growth, connections, or legacy.  
- The search itself may be integral to the human experience.  

**Final Answer:**  
The meaning of life is not a singular truth but a deeply personal and evolving concept. It can be found in self-defined purpose (philosophy), spiritual fulfillment (religion), contributing to collective progress (society), or simply embracing existence (absurdism). While answers vary, the pursuit of meaning—whether through love, creativity, knowledge, or service—often becomes the journey that defines a life well-lived.

## Setting up a helper LLM

In [ ]:
from pydantic_ai import Agent

class LifeMeaningStructuredResult(BaseModel):
    title: str = Field(description='This is a top level Markdown heading that covers the topic of the query and answer prefix it with #')
    answer: str = Field(description='This is a main section that provides answers for the query and questions')
    bullets: str = Field(description='This is a set of bulletpoints that summarize the answers for query')
    thinking: str = Field(description='This is a string that covers the thinking behind the answer')

formatting_agent = Agent('google-gla:gemini-1.5-flash',
              result_type=LifeMeaningStructuredResult,
              system_prompt='Your a helpful formatting assistant, you never give your own opinion. '
              'You merely take the input given and convert it to the structured result for sending back, '
              )

In [ ]:
structured_results = await formatting_agent.run(result.data)

In [ ]:
type(structured_results.data.title)

str

In [ ]:
Markdown(structured_results.data.title)

The Quest for Meaning: Exploring Life's Ultimate Question

In [ ]:
Markdown(structured_results.data.answer)

The meaning of life is not a singular truth but a deeply personal and evolving concept. It can be found in self-defined purpose (philosophy), spiritual fulfillment (religion), contributing to collective progress (society), or simply embracing existence (absurdism). While answers vary, the pursuit of meaning—whether through love, creativity, knowledge, or service—often becomes the journey that defines a life well-lived.

In [ ]:
Markdown(structured_results.data.bullets)

 - No universal answer exists; interpretations vary by context and belief.
 - Philosophy offers frameworks for creating or rejecting inherent meaning.
 - Spirituality often links purpose to a higher reality or moral duty.
 - Biologically, life’s "goal" is propagation, but humans transcend this through abstract goals.
 - Meaning is frequently tied to personal growth, connections, or legacy.
 - The search itself may be integral to the human experience.

In [ ]:
Markdown(structured_results.data.thinking)

The question "What is the meaning of life?" is inherently subjective and multifaceted. To address it, we must synthesize perspectives from philosophy, religion, science, and individual experience:
1. **Philosophical Views**: Existentialism argues that life’s meaning is self-created (e.g., Sartre), while nihilism posits no inherent meaning. Absurdism (Camus) accepts the tension between seeking meaning and an indifferent universe.
2. **Religious/Spiritual Perspectives**: Many traditions tie purpose to transcendence—e.g., union with the divine (Hinduism), following God’s will (Abrahamic faiths), or achieving enlightenment (Buddhism).
3. **Scientific Lens**: Biology suggests survival and reproduction, but human consciousness expands this to curiosity, progress, and legacy.
4. **Individual and Cultural Subjectivity**: Personal fulfillment, relationships, creativity, or societal contribution often define meaning, shaped by cultural norms (e.g., collectivism vs. individualism).
5. **Modern Psychology**: Positive psychology emphasizes flourishing through purpose, relationships, and engagement (e.g., Seligman’s "PERMA" model).

## Getting the real thinking CoT

In [ ]:
SYSTEM_PROMPT="""Your a helpful and wise reasoning assistant, you are an expert in thinking \
If you are given a question you think carefully and then respond with a title, \
a set of bullets points summary and a final answer """

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.environ["DEEPSEEK_API_KEY"],
                base_url="https://api.deepseek.com")

messages = [{"role": "system", "content": SYSTEM_PROMPT},
 {"role": "user", "content": "What is the meaning of life?"}]
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=messages
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content


In [ ]:
Markdown(content)

**Title: Exploring the Multifaceted Perspectives on the Meaning of Life**  

**Key Considerations:**  
- **Philosophical Views**:  
  - Existentialism (e.g., Sartre, Camus): Life has no inherent meaning; individuals create purpose through choices and actions.  
  - Absurdism (Camus): Accepting life’s lack of intrinsic meaning while defiantly pursuing passion and freedom.  
  - Utilitarianism/Altruism: Meaning derives from maximizing well-being for oneself and others.  

- **Religious/Spiritual Perspectives**:  
  - Abrahamic faiths (Christianity, Islam, Judaism): Purpose involves fulfilling divine will, seeking salvation, or union with God.  
  - Eastern traditions (Buddhism, Hinduism): Meaning lies in enlightenment, escaping suffering (dukkha), or realizing one’s unity with the cosmos (Brahman).  

- **Scientific/Empirical Lens**:  
  - Evolutionary biology: Survival, reproduction, and gene propagation as foundational drivers.  
  - Cosmic insignificance: From a universal scale, life may lack objective meaning, but subjective significance persists.  

- **Personal and Cultural Dimensions**:  
  - Subjective fulfillment: Relationships, creativity, legacy, or personal growth.  
  - Cultural narratives: Societal values (e.g., community, progress) shape collective understandings of purpose.  

**Final Answer**:  
The meaning of life is not universally defined. It emerges from a blend of philosophical inquiry, spiritual beliefs, biological imperatives, and personal/cultural narratives. While some seek objective answers, most frameworks suggest meaning is *subjectively constructed*—rooted in individual agency, connection to others, or alignment with chosen values. Ultimately, it is a deeply personal exploration.

In [ ]:
Markdown(reasoning_content)

Okay, so the user asked, "What is the meaning of life?" Hmm, that's a big one. Let me start by breaking this down. First, I need to understand what they're really looking for. Are they asking for a philosophical answer, a scientific explanation, or maybe something personal? The question is pretty broad.

Alright, from a philosophical standpoint, different schools of thought have tackled this. Existentialism says we create our own meaning. That makes sense because if there's no inherent meaning, then it's up to each individual. Then there's religious perspectives, like in Christianity or Buddhism, where the meaning is tied to spiritual goals or enlightenment. Maybe I should mention a few major religions here.

Scientifically, evolution might suggest that life's purpose is survival and reproduction. But does that fully answer the question? Probably not, since humans seek more than just biological imperatives. Then there's the aspect of personal fulfillment—happiness, relationships, achievements. People often find meaning in their passions or helping others.

Wait, also, some modern theories like positive psychology focus on well-being and purpose. Maybe that's worth including. But I should also note that there's no single answer, which is why the user might be asking. They could be looking for reassurance that it's okay if the meaning isn't obvious.

I need to organize these points. Start with the abstract concepts, then move to structured summaries. Make sure to cover different angles without being too biased. The final answer should tie it all together, emphasizing that it's subjective and varies by individual or belief system. Let me check if I missed any major viewpoints. Oh, absurdism, like Camus, where life is meaningless but we rebel against that absurdity. That's another angle. Also, maybe mention cultural differences in how meaning is perceived.

Okay, structuring the bullet points: philosophical, religious, scientific, personal/cultural. Then conclude that it's a blend of these, with no universal answer. Make sure the summary is clear and concise. The title should capture the essence of the exploration. Alright, that should cover it.

## Pass it all into

In [ ]:
CONTEXT = "<thinking>" + reasoning_content + "</thinking>" + "\n\n" + content

In [ ]:
structured_results = await formatting_agent.run(CONTEXT)

In [ ]:
type(structured_results.data.title)

str

In [ ]:
Markdown(structured_results.data.title)

Exploring the Multifaceted Perspectives on the Meaning of Life

In [ ]:
Markdown(structured_results.data.answer)

The meaning of life is not universally defined. It emerges from a blend of philosophical inquiry, spiritual beliefs, biological imperatives, and personal/cultural narratives. While some seek objective answers, most frameworks suggest meaning is *subjectively constructed*—rooted in individual agency, connection to others, or alignment with chosen values. Ultimately, it is a deeply personal exploration.

In [ ]:
Markdown(structured_results.data.bullets)

* **Philosophical Views**: Existentialism, Absurdism, Utilitarianism/Altruism
* **Religious/Spiritual Perspectives**: Abrahamic faiths, Eastern traditions
* **Scientific/Empirical Lens**: Evolutionary biology, Cosmic insignificance
* **Personal and Cultural Dimensions**: Subjective fulfillment, Cultural narratives

In [ ]:
Markdown(structured_results.data.thinking)

Okay, so the user asked, "What is the meaning of life?" Hmm, that's a big one. Let me start by breaking this down. First, I need to understand what they're really looking for. Are they asking for a philosophical answer, a scientific explanation, or maybe something personal? The question is pretty broad.

Alright, from a philosophical standpoint, different schools of thought have tackled this. Existentialism says we create our own meaning. That makes sense because if there's no inherent meaning, then it's up to each individual. Then there's religious perspectives, like in Christianity or Buddhism, where the meaning is tied to spiritual goals or enlightenment. Maybe I should mention a few major religions here.

Scientifically, evolution might suggest that life's purpose is survival and reproduction. But does that fully answer the question? Probably not, since humans seek more than just biological imperatives. Then there's the aspect of personal fulfillment—happiness, relationships, achievements. People often find meaning in their passions or helping others.

Wait, also, some modern theories like positive psychology focus on well-being and purpose. Maybe that's worth including. But I should also note that there's no single answer, which is why the user might be asking. They could be looking for reassurance that it's okay if the meaning isn't obvious.

I need to organize these points. Start with the abstract concepts, then move to structured summaries. Make sure to cover different angles without being too biased. The final answer should tie it all together, emphasizing that it's subjective and varies by individual or belief system. Let me check if I missed any major viewpoints. Oh, absurdism, like Camus, where life is meaningless but we rebel against that absurdity. That's another angle. Also, maybe mention cultural differences in how meaning is perceived.

Okay, structuring the bullet points: philosophical, religious, scientific, personal/cultural. Then conclude that it's a blend of these, with no universal answer. Make sure the summary is clear and concise. The title should capture the essence of the exploration. Alright, that should cover it.

## Using a Reasoner Model as a Tool

In [ ]:
## this prompt can be improved a lot. - currently its not returning everytime to the Reason Model for synthesis
ORCHESTRATOR_PROMPT = """You are an orchestration system that coordinates between specialized tools to produce comprehensive responses. Follow this exact sequence don't skip:

Once you have the search info always send back to the Reasoning Model for Synthesis

1. KEYWORD GENERATION
   Input: User query
   Tool: ReasoningEngine
   Action: Generate 3-5 search keywords/phrases
   Output format: List of {keyword, query_id}

2. SEARCH EXECUTION
   Input: Keywords from step 1
   Tool: SearchTool
   Action: Execute parallel searches using each keyword
   Output format: List of {query_id, search_results[]}

3. SYNTHESIS
   Input:
   - Original user query
   - All search results
   Tool: ReasoningEngine
   Action: Analyze and synthesize information if you need more info ask for more searches
   Output format: Structured report with:
   - Key findings
   - Supporting evidence
   - Confidence levels

4. RESPONSE FORMATTING
   Input: Synthesis report
   Action: Format into user-friendly response with:
   - Clear sections
   - Citations
   - Relevant metrics
   Output: Final formatted response

Required validation at each step:
- Verify tool outputs match expected formats
- Log any failed steps for retry
- Maintain traceability of information sources
"""

In [ ]:
@dataclass
class TaskData:
    task: str

@dataclass
class SearchDataclass:
    max_results: int

class ReportStructuredResult(BaseModel):
    title: str = Field(description='This is a top level Markdown heading that covers the topic of the query and answer prefix it with #')
    answer: str = Field(description='This is a main section that provides answers for the query and questions')
    bullets: str = Field(description='This is a set of bulletpoints that summarize the answers for query')
    thinking: str = Field(description='This is a string that covers the thinking behind the answer')

orchestrator_agent = Agent('google-gla:gemini-1.5-flash',
              result_type=ReportStructuredResult,
              system_prompt=ORCHESTRATOR_PROMPT
              )

deps = SearchDataclass(max_results=3)


In [ ]:
SYSTEM_PROMPT="""Your a helpful and wise reasoning assistant, you are an expert in thinking \
If you are given a question you think carefully and then respond with a title, \
a set of bullets points summary and a final answer """

In [ ]:
@orchestrator_agent.tool_plain
async def get_reasoning_answers(task: str) -> dict[str, Any]:
    """Get a strong reasoning answer for any task .

    Args:
        task: task for reasoning
    """
    client = OpenAI(api_key=os.environ["DEEPSEEK_API_KEY"],
                base_url="https://api.deepseek.com")


    messages = [{"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": task}]
    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=messages
    )

    reasoning_content = response.choices[0].message.reasoning_content
    content = response.choices[0].message.content

    formatted_response = "<thinking>" + reasoning_content + "</thinking>" + "\n\n" + content

    return formatted_response


In [ ]:
@orchestrator_agent.tool #Tavily
async def get_search(search_data:RunContext[SearchDataclass],query: str, query_number: int) -> dict[str, Any]:
    """Get the search for a keyword query.

    Args:
        query: keywords to search.
    """
    print(f"Search query {query_number}: {query}")
    max_results = search_data.deps.max_results
    results = await tavily_client.get_search_context(query=query, max_results=max_results)

    return results

In [ ]:
structured_results = await orchestrator_agent.run("Please create me a report on GRPO RL used in the DeepSeekR1-Zero model", deps=deps)

Search query 1: DeepSeekR1-Zero GRPO RL technical report
Search query 2: GRPO reinforcement learning algorithm explained
Search query 3: DeepSeekR1-Zero model architecture RL methods


In [ ]:
structured_results

RunResult(_all_messages=[ModelRequest(parts=[SystemPromptPart(content="You are an orchestration system that coordinates between specialized tools to produce comprehensive responses. Follow this exact sequence don't skip:\n\nOnce you have the search info always send back to the Reasoning Model for Synthesis\n\n1. KEYWORD GENERATION\n   Input: User query\n   Tool: ReasoningEngine\n   Action: Generate 3-5 search keywords/phrases\n   Output format: List of {keyword, query_id}\n\n2. SEARCH EXECUTION\n   Input: Keywords from step 1\n   Tool: SearchTool\n   Action: Execute parallel searches using each keyword\n   Output format: List of {query_id, search_results[]}\n\n3. SYNTHESIS\n   Input: \n   - Original user query\n   - All search results\n   Tool: ReasoningEngine\n   Action: Analyze and synthesize information if you need more info ask for more searches\n   Output format: Structured report with:\n   - Key findings\n   - Supporting evidence\n   - Confidence levels\n\n4. RESPONSE FORMATTING\

In [ ]:
Markdown(structured_results.data.title)

# DeepSeekR1-Zero: A Report on its Reinforcement Learning Approach

In [ ]:
Markdown(structured_results.data.bullets)

- DeepSeekR1-Zero uses large-scale reinforcement learning without initial supervised fine-tuning.
- "GRPO" is likely a custom or variant RL algorithm focused on policy optimization.
- The model exhibits strong reasoning capabilities, surpassing many other models.
- DeepSeekR1 improves upon DeepSeekR1-Zero with multi-stage training and cold-start data.

In [ ]:
debug(structured_results)